### Creation of index with custom mappings failed so using csv- datatype conversion method

In [2]:
import requests
import json
from elasticsearch import Elasticsearch
import pprint

## Elasticsearch python client


In [3]:

es = Elasticsearch('http://localhost:9200')
print('done')


done


In [4]:
es

<Elasticsearch(['http://localhost:9200'])>

## checking mystoredata index

In [7]:
%%time
home= requests.get(url = "http://localhost:9200/mystoredata")
# print(home.content)
pprint.pprint(json.loads(home.text))

{'mystoredata': {'aliases': {},
                 'mappings': {'properties': {'Item_Fat_Content': {'fields': {'keyword': {'ignore_above': 256,
                                                                                         'type': 'keyword'}},
                                                                  'type': 'text'},
                                             'Item_Identifier': {'fields': {'keyword': {'ignore_above': 256,
                                                                                        'type': 'keyword'}},
                                                                 'type': 'text'},
                                             'Item_MRP': {'fields': {'keyword': {'ignore_above': 256,
                                                                                 'type': 'keyword'}},
                                                          'type': 'text'},
                                             'Item_ManufactureDate': {'type': 'date'},

## First -  prepare dataset

In [8]:
dataPath = "datasets_notebooks/archive (3)/Train.csv"


In [9]:
import csv

In [10]:
%%time
dataList = []
# each row in dictreader is a dictionary field : item
print(len(dataList))

with open(dataPath , 'r') as d:
    csvDict = csv.DictReader(d)
    for rows in csvDict:
        dataList.append(rows)
    print('done')
    
print(len(dataList))

0
done
8523
CPU times: user 43.4 ms, sys: 8.01 ms, total: 51.4 ms
Wall time: 51 ms


In [11]:
for keys in dataList[0]:
    print(keys ,' -> ',dataList[0][keys] , type(dataList[0][keys]))

Item_Identifier  ->  FDA15 <class 'str'>
Item_Weight  ->  9.3 <class 'str'>
Item_Fat_Content  ->  Low Fat <class 'str'>
Item_Visibility  ->  0.016047301 <class 'str'>
Item_Type  ->  Dairy <class 'str'>
Item_MRP  ->  249.8092 <class 'str'>
Outlet_Identifier  ->  OUT049 <class 'str'>
Outlet_Establishment_Year  ->  1999 <class 'str'>
Outlet_Size  ->  Medium <class 'str'>
Outlet_Location_Type  ->  Tier 1 <class 'str'>
Outlet_Type  ->  Supermarket Type1 <class 'str'>
Item_Outlet_Sales  ->  3735.138 <class 'str'>


## evrything is read as string convert to correct datatype first

In [12]:
from datetime import datetime , timedelta
import random

In [13]:
sampleDate = datetime(2010,5,11)
# adding a date type field in index

In [14]:
dateGen = lambda daysDiff : sampleDate + timedelta(days = daysDiff)


In [16]:
for i in range(100,300,15):
    print(i, 'days after ',sampleDate.date())
#     newDate = dateGen(i)
    print('\t\t-> ',{'Item_ManufactureDate' :dateGen(i).date().strftime('%Y-%m-%d') })

100 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-08-19'}
115 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-09-03'}
130 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-09-18'}
145 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-10-03'}
160 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-10-18'}
175 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-11-02'}
190 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-11-17'}
205 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-12-02'}
220 days after  2010-05-11
		->  {'Item_ManufactureDate': '2010-12-17'}
235 days after  2010-05-11
		->  {'Item_ManufactureDate': '2011-01-01'}
250 days after  2010-05-11
		->  {'Item_ManufactureDate': '2011-01-16'}
265 days after  2010-05-11
		->  {'Item_ManufactureDate': '2011-01-31'}
280 days after  2010-05-11
		->  {'Item_ManufactureDate': '2011-02-15'}
295 days after  2010-05-11
		->  {'Item_ManufactureDate': '2011-

In [17]:
%%time
manufacturingDatesDict = [{'Item_ManufactureDate' :dateGen(i).date().strftime('%Y-%m-%d') } for i in range(50,500,20)]
len(manufacturingDatesDict)

CPU times: user 455 µs, sys: 43 µs, total: 498 µs
Wall time: 424 µs


23

In [21]:
dataInsert = []


In [22]:
convertToFloat = lambda x : 0.0 if len(x)<1 else float(x)

In [23]:
%%time
# copy should prevent updation of default dictionary

for index,d in enumerate(dataList):
    # everything is read as string --- convert for each key with correct datatype
    print(f'running for index = {index}')
    newDict = {}
    newDict['Item_Identifier'] = d['Item_Identifier']
    newDict['Item_Weight'] = convertToFloat(d['Item_Weight'])
    newDict['Item_Fat_Content'] = d['Item_Fat_Content']
    newDict['Item_Visibility'] = convertToFloat(d['Item_Visibility'])
    newDict['Item_Type'] = d['Item_Type']
    newDict['Item_MRP'] = convertToFloat(d['Item_MRP'])
    newDict['Outlet_Identifier'] = d['Outlet_Identifier']
    newDict['Outlet_Location_Type'] = d['Outlet_Location_Type']
    newDict['Outlet_Type'] = d['Outlet_Type']
    newDict['Item_ManufactureDate'] = random.choice(manufacturingDatesDict)['Item_ManufactureDate']
    
    dataInsert.append(newDict)
    del newDict
print('done')

running for index = 0
running for index = 1
running for index = 2
running for index = 3
running for index = 4
running for index = 5
running for index = 6
running for index = 7
running for index = 8
running for index = 9
running for index = 10
running for index = 11
running for index = 12
running for index = 13
running for index = 14
running for index = 15
running for index = 16
running for index = 17
running for index = 18
running for index = 19
running for index = 20
running for index = 21
running for index = 22
running for index = 23
running for index = 24
running for index = 25
running for index = 26
running for index = 27
running for index = 28
running for index = 29
running for index = 30
running for index = 31
running for index = 32
running for index = 33
running for index = 34
running for index = 35
running for index = 36
running for index = 37
running for index = 38
running for index = 39
running for index = 40
running for index = 41
running for index = 42
running for index = 4

In [24]:
dataInsert[0]

{'Item_Identifier': 'FDA15',
 'Item_Weight': 9.3,
 'Item_Fat_Content': 'Low Fat',
 'Item_Visibility': 0.016047301,
 'Item_Type': 'Dairy',
 'Item_MRP': 249.8092,
 'Outlet_Identifier': 'OUT049',
 'Outlet_Location_Type': 'Tier 1',
 'Outlet_Type': 'Supermarket Type1',
 'Item_ManufactureDate': '2011-06-25'}

## Append data to index ( create new index -- myfifthstore )

In [26]:
es = Elasticsearch('http://localhost:9200')
print('done')
es

done


<Elasticsearch(['http://localhost:9200'])>

## index created without mappings

In [27]:
%%time

responseCreate = es.indices.create(index='myfifthstore')
print(responseCreate)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'myfifthstore'}
CPU times: user 71.7 ms, sys: 12.2 ms, total: 84 ms
Wall time: 198 ms


<timed exec>:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.


## sample insert with 0 index data from dataInsert list

In [28]:
sampleData = dataInsert[0]

In [29]:
sampleData

{'Item_Identifier': 'FDA15',
 'Item_Weight': 9.3,
 'Item_Fat_Content': 'Low Fat',
 'Item_Visibility': 0.016047301,
 'Item_Type': 'Dairy',
 'Item_MRP': 249.8092,
 'Outlet_Identifier': 'OUT049',
 'Outlet_Location_Type': 'Tier 1',
 'Outlet_Type': 'Supermarket Type1',
 'Item_ManufactureDate': '2011-06-25'}

# first entry will define mapping

In [30]:
%%time
home= requests.get(url = "http://localhost:9200/myfifthstore")
# print(home.content)
pprint.pprint(json.loads(home.text))

{'myfifthstore': {'aliases': {},
                  'mappings': {},
                  'settings': {'index': {'creation_date': '1655803718672',
                                         'number_of_replicas': '1',
                                         'number_of_shards': '1',
                                         'provided_name': 'myfifthstore',
                                         'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
                                         'uuid': '82bYIvz5TaGgL_NP-Ek-kQ',
                                         'version': {'created': '7170499'}}}}}
CPU times: user 4.99 ms, sys: 3.35 ms, total: 8.34 ms
Wall time: 13.5 ms


##### currently mappings is empty

### and index is empty

In [33]:
%%time
# allIndices= requests.get(url = "http://localhost:9200/_aliases")
allIndices= requests.get(url = "http://localhost:9200/myfifthstore/_search?pretty=true&q=*:*")
# print(home.content)
pprint.pprint(allIndices.text)



('{\n'
 '  "took" : 19,\n'
 '  "timed_out" : false,\n'
 '  "_shards" : {\n'
 '    "total" : 1,\n'
 '    "successful" : 1,\n'
 '    "skipped" : 0,\n'
 '    "failed" : 0\n'
 '  },\n'
 '  "hits" : {\n'
 '    "total" : {\n'
 '      "value" : 0,\n'
 '      "relation" : "eq"\n'
 '    },\n'
 '    "max_score" : null,\n'
 '    "hits" : [ ]\n'
 '  }\n'
 '}\n')
CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 41 ms


In [34]:
%%time

es.index(index='myfifthstore', id=0,document=sampleData)

CPU times: user 7.02 ms, sys: 4.05 ms, total: 11.1 ms
Wall time: 65.3 ms


<timed eval>:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.


ObjectApiResponse({'_index': 'myfifthstore', '_type': '_doc', '_id': '0', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [35]:
%%time
home= requests.get(url = "http://localhost:9200/myfifthstore")
# print(home.content)
pprint.pprint(json.loads(home.text))

{'myfifthstore': {'aliases': {},
                  'mappings': {'properties': {'Item_Fat_Content': {'fields': {'keyword': {'ignore_above': 256,
                                                                                          'type': 'keyword'}},
                                                                   'type': 'text'},
                                              'Item_Identifier': {'fields': {'keyword': {'ignore_above': 256,
                                                                                         'type': 'keyword'}},
                                                                  'type': 'text'},
                                              'Item_MRP': {'type': 'float'},
                                              'Item_ManufactureDate': {'type': 'date'},
                                              'Item_Type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                   'type'

## seeing data in index

In [37]:
%%time
# allIndices= requests.get(url = "http://localhost:9200/_aliases")
allIndices= requests.get(url = "http://localhost:9200/myfifthstore/_search?q=*:*")
# print(home.content)
pprint.pprint(allIndices.text)



('{"took":2,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":1,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"myfifthstore","_type":"_doc","_id":"0","_score":1.0,"_source":{"Item_Identifier":"FDA15","Item_Weight":9.3,"Item_Fat_Content":"Low '
 'Fat","Item_Visibility":0.016047301,"Item_Type":"Dairy","Item_MRP":249.8092,"Outlet_Identifier":"OUT049","Outlet_Location_Type":"Tier '
 '1","Outlet_Type":"Supermarket '
 'Type1","Item_ManufactureDate":"2011-06-25"}}]}}')
CPU times: user 5.38 ms, sys: 380 µs, total: 5.76 ms
Wall time: 9.38 ms


### json.dump with sample data --- see if it fixes datatypes

## looping an pushing all data into myfifthstore then visualize data in kibana

#### start from index 1

In [42]:
%%time

for index,document in enumerate(dataInsert):
    if index==0:
        print('already inserted')
    else:
        es.index(index='myfifthstore', id=index,document=document)
        print(f" indexed = {index} ")
print('\n\ndone')

already inserted
 indexed = 1 
 indexed = 2 
 indexed = 3 
 indexed = 4 
 indexed = 5 
 indexed = 6 
 indexed = 7 
 indexed = 8 
 indexed = 9 
 indexed = 10 
 indexed = 11 
 indexed = 12 


<timed exec>:5: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.


 indexed = 13 
 indexed = 14 
 indexed = 15 
 indexed = 16 
 indexed = 17 
 indexed = 18 
 indexed = 19 
 indexed = 20 
 indexed = 21 
 indexed = 22 
 indexed = 23 
 indexed = 24 
 indexed = 25 
 indexed = 26 
 indexed = 27 
 indexed = 28 
 indexed = 29 
 indexed = 30 
 indexed = 31 
 indexed = 32 
 indexed = 33 
 indexed = 34 
 indexed = 35 
 indexed = 36 
 indexed = 37 
 indexed = 38 
 indexed = 39 
 indexed = 40 
 indexed = 41 
 indexed = 42 
 indexed = 43 
 indexed = 44 
 indexed = 45 
 indexed = 46 
 indexed = 47 
 indexed = 48 
 indexed = 49 
 indexed = 50 
 indexed = 51 
 indexed = 52 
 indexed = 53 
 indexed = 54 
 indexed = 55 
 indexed = 56 
 indexed = 57 
 indexed = 58 
 indexed = 59 
 indexed = 60 
 indexed = 61 
 indexed = 62 
 indexed = 63 
 indexed = 64 
 indexed = 65 
 indexed = 66 
 indexed = 67 
 indexed = 68 
 indexed = 69 
 indexed = 70 
 indexed = 71 
 indexed = 72 
 indexed = 73 
 indexed = 74 
 indexed = 75 
 indexed = 76 
 indexed = 77 
 indexed = 78 
 indexed =

 indexed = 547 
 indexed = 548 
 indexed = 549 
 indexed = 550 
 indexed = 551 
 indexed = 552 
 indexed = 553 
 indexed = 554 
 indexed = 555 
 indexed = 556 
 indexed = 557 
 indexed = 558 
 indexed = 559 
 indexed = 560 
 indexed = 561 
 indexed = 562 
 indexed = 563 
 indexed = 564 
 indexed = 565 
 indexed = 566 
 indexed = 567 
 indexed = 568 
 indexed = 569 
 indexed = 570 
 indexed = 571 
 indexed = 572 
 indexed = 573 
 indexed = 574 
 indexed = 575 
 indexed = 576 
 indexed = 577 
 indexed = 578 
 indexed = 579 
 indexed = 580 
 indexed = 581 
 indexed = 582 
 indexed = 583 
 indexed = 584 
 indexed = 585 
 indexed = 586 
 indexed = 587 
 indexed = 588 
 indexed = 589 
 indexed = 590 
 indexed = 591 
 indexed = 592 
 indexed = 593 
 indexed = 594 
 indexed = 595 
 indexed = 596 
 indexed = 597 
 indexed = 598 
 indexed = 599 
 indexed = 600 
 indexed = 601 
 indexed = 602 
 indexed = 603 
 indexed = 604 
 indexed = 605 
 indexed = 606 
 indexed = 607 
 indexed = 608 
 indexed

 indexed = 1062 
 indexed = 1063 
 indexed = 1064 
 indexed = 1065 
 indexed = 1066 
 indexed = 1067 
 indexed = 1068 
 indexed = 1069 
 indexed = 1070 
 indexed = 1071 
 indexed = 1072 
 indexed = 1073 
 indexed = 1074 
 indexed = 1075 
 indexed = 1076 
 indexed = 1077 
 indexed = 1078 
 indexed = 1079 
 indexed = 1080 
 indexed = 1081 
 indexed = 1082 
 indexed = 1083 
 indexed = 1084 
 indexed = 1085 
 indexed = 1086 
 indexed = 1087 
 indexed = 1088 
 indexed = 1089 
 indexed = 1090 
 indexed = 1091 
 indexed = 1092 
 indexed = 1093 
 indexed = 1094 
 indexed = 1095 
 indexed = 1096 
 indexed = 1097 
 indexed = 1098 
 indexed = 1099 
 indexed = 1100 
 indexed = 1101 
 indexed = 1102 
 indexed = 1103 
 indexed = 1104 
 indexed = 1105 
 indexed = 1106 
 indexed = 1107 
 indexed = 1108 
 indexed = 1109 
 indexed = 1110 
 indexed = 1111 
 indexed = 1112 
 indexed = 1113 
 indexed = 1114 
 indexed = 1115 
 indexed = 1116 
 indexed = 1117 
 indexed = 1118 
 indexed = 1119 
 indexed = 112

...

 indexed = 6470 
 indexed = 6471 
 indexed = 6472 
 indexed = 6473 
 indexed = 6474 
 indexed = 6475 
 indexed = 6476 
 indexed = 6477 
 indexed = 6478 
 indexed = 6479 
 indexed = 6480 
 indexed = 6481 
 indexed = 6482 
 indexed = 6483 
 indexed = 6484 
 indexed = 6485 
 indexed = 6486 
 indexed = 6487 
 indexed = 6488 
 indexed = 6489 
 indexed = 6490 
 indexed = 6491 
 indexed = 6492 
 indexed = 6493 
 indexed = 6494 
 indexed = 6495 
 indexed = 6496 
 indexed = 6497 
 indexed = 6498 
 indexed = 6499 
 indexed = 6500 
 indexed = 6501 
 indexed = 6502 
 indexed = 6503 
 indexed = 6504 
 indexed = 6505 
 indexed = 6506 
 indexed = 6507 
 indexed = 6508 
 indexed = 6509 
 indexed = 6510 
 indexed = 6511 
 indexed = 6512 
 indexed = 6513 
 indexed = 6514 
 indexed = 6515 
 indexed = 6516 
 indexed = 6517 
 indexed = 6518 
 indexed = 6519 
 indexed = 6520 
 indexed = 6521 
 indexed = 6522 
 indexed = 6523 
 indexed = 6524 
 indexed = 6525 
 indexed = 6526 
 indexed = 6527 
 indexed = 652

 indexed = 6952 
 indexed = 6953 
 indexed = 6954 
 indexed = 6955 
 indexed = 6956 
 indexed = 6957 
 indexed = 6958 
 indexed = 6959 
 indexed = 6960 
 indexed = 6961 
 indexed = 6962 
 indexed = 6963 
 indexed = 6964 
 indexed = 6965 
 indexed = 6966 
 indexed = 6967 
 indexed = 6968 
 indexed = 6969 
 indexed = 6970 
 indexed = 6971 
 indexed = 6972 
 indexed = 6973 
 indexed = 6974 
 indexed = 6975 
 indexed = 6976 
 indexed = 6977 
 indexed = 6978 
 indexed = 6979 
 indexed = 6980 
 indexed = 6981 
 indexed = 6982 
 indexed = 6983 
 indexed = 6984 
 indexed = 6985 
 indexed = 6986 
 indexed = 6987 
 indexed = 6988 
 indexed = 6989 
 indexed = 6990 
 indexed = 6991 
 indexed = 6992 
 indexed = 6993 
 indexed = 6994 
 indexed = 6995 
 indexed = 6996 
 indexed = 6997 
 indexed = 6998 
 indexed = 6999 
 indexed = 7000 
 indexed = 7001 
 indexed = 7002 
 indexed = 7003 
 indexed = 7004 
 indexed = 7005 
 indexed = 7006 
 indexed = 7007 
 indexed = 7008 
 indexed = 7009 
 indexed = 701

 indexed = 7440 
 indexed = 7441 
 indexed = 7442 
 indexed = 7443 
 indexed = 7444 
 indexed = 7445 
 indexed = 7446 
 indexed = 7447 
 indexed = 7448 
 indexed = 7449 
 indexed = 7450 
 indexed = 7451 
 indexed = 7452 
 indexed = 7453 
 indexed = 7454 
 indexed = 7455 
 indexed = 7456 
 indexed = 7457 
 indexed = 7458 
 indexed = 7459 
 indexed = 7460 
 indexed = 7461 
 indexed = 7462 
 indexed = 7463 
 indexed = 7464 
 indexed = 7465 
 indexed = 7466 
 indexed = 7467 
 indexed = 7468 
 indexed = 7469 
 indexed = 7470 
 indexed = 7471 
 indexed = 7472 
 indexed = 7473 
 indexed = 7474 
 indexed = 7475 
 indexed = 7476 
 indexed = 7477 
 indexed = 7478 
 indexed = 7479 
 indexed = 7480 
 indexed = 7481 
 indexed = 7482 
 indexed = 7483 
 indexed = 7484 
 indexed = 7485 
 indexed = 7486 
 indexed = 7487 
 indexed = 7488 
 indexed = 7489 
 indexed = 7490 
 indexed = 7491 
 indexed = 7492 
 indexed = 7493 
 indexed = 7494 
 indexed = 7495 
 indexed = 7496 
 indexed = 7497 
 indexed = 749

 indexed = 7935 
 indexed = 7936 
 indexed = 7937 
 indexed = 7938 
 indexed = 7939 
 indexed = 7940 
 indexed = 7941 
 indexed = 7942 
 indexed = 7943 
 indexed = 7944 
 indexed = 7945 
 indexed = 7946 
 indexed = 7947 
 indexed = 7948 
 indexed = 7949 
 indexed = 7950 
 indexed = 7951 
 indexed = 7952 
 indexed = 7953 
 indexed = 7954 
 indexed = 7955 
 indexed = 7956 
 indexed = 7957 
 indexed = 7958 
 indexed = 7959 
 indexed = 7960 
 indexed = 7961 
 indexed = 7962 
 indexed = 7963 
 indexed = 7964 
 indexed = 7965 
 indexed = 7966 
 indexed = 7967 
 indexed = 7968 
 indexed = 7969 
 indexed = 7970 
 indexed = 7971 
 indexed = 7972 
 indexed = 7973 
 indexed = 7974 
 indexed = 7975 
 indexed = 7976 
 indexed = 7977 
 indexed = 7978 
 indexed = 7979 
 indexed = 7980 
 indexed = 7981 
 indexed = 7982 
 indexed = 7983 
 indexed = 7984 
 indexed = 7985 
 indexed = 7986 
 indexed = 7987 
 indexed = 7988 
 indexed = 7989 
 indexed = 7990 
 indexed = 7991 
 indexed = 7992 
 indexed = 799

 indexed = 8429 
 indexed = 8430 
 indexed = 8431 
 indexed = 8432 
 indexed = 8433 
 indexed = 8434 
 indexed = 8435 
 indexed = 8436 
 indexed = 8437 
 indexed = 8438 
 indexed = 8439 
 indexed = 8440 
 indexed = 8441 
 indexed = 8442 
 indexed = 8443 
 indexed = 8444 
 indexed = 8445 
 indexed = 8446 
 indexed = 8447 
 indexed = 8448 
 indexed = 8449 
 indexed = 8450 
 indexed = 8451 
 indexed = 8452 
 indexed = 8453 
 indexed = 8454 
 indexed = 8455 
 indexed = 8456 
 indexed = 8457 
 indexed = 8458 
 indexed = 8459 
 indexed = 8460 
 indexed = 8461 
 indexed = 8462 
 indexed = 8463 
 indexed = 8464 
 indexed = 8465 
 indexed = 8466 
 indexed = 8467 
 indexed = 8468 
 indexed = 8469 
 indexed = 8470 
 indexed = 8471 
 indexed = 8472 
 indexed = 8473 
 indexed = 8474 
 indexed = 8475 
 indexed = 8476 
 indexed = 8477 
 indexed = 8478 
 indexed = 8479 
 indexed = 8480 
 indexed = 8481 
 indexed = 8482 
 indexed = 8483 
 indexed = 8484 
 indexed = 8485 
 indexed = 8486 
 indexed = 848